
# 文件加载与预览



In [4]:
import sys
import json
import math
import numbers
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# set up display area to show dataframe in jupyter qtconsole

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
# 使用notepad++ 修改文件格式和编码：utf-8
path_train = r'train.csv'
path_test = r'test.csv'

##文件不规则，行尾有分隔符，则可以设定index_col=False 来是的pandas不适用第一列作为行索引。
train_pdf = pd.read_csv(path_train,encoding='utf-8',engine='python',dtype='object',index_col=False)
test_pdf = pd.read_csv(path_test,encoding='utf-8',engine='python',dtype='object')

pd.set_option('max_columns',1000) 
pd.set_option('max_row',300) 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
train_pdf.head(10)

,Unnamed: 0,分类,文章,字数
0,0,娱乐,《青蛇》造型师默认新《红楼梦》额妆抄袭（图） 凡是看过电影《青蛇》的人，都不会忘记青白二蛇的...,1519
1,1,娱乐,６．１６日剧榜 ＜最后的朋友＞ 亮最后杀招成功登顶 《最后的朋友》本周的电视剧排行榜单依然只...,2016
2,2,娱乐,超乎想象的好看《纳尼亚传奇２：凯斯宾王子》 现时资讯如此发达，搜狐电影评审团几乎人人在没有看...,1770
3,3,娱乐,吴宇森：赤壁大战不会出现在上集 “希望《赤壁》能给你们不一样的感觉。”对于自己刚刚拍完的影片...,1344
4,4,娱乐,组图：《多情女人痴情男》陈浩民现场耍宝 陈浩民：外面的朋友大家好，现在是搜狐现场直播，欢迎《...,3216
5,5,娱乐,艺人涉毒被警方查事件回顾 正方观点：相信：人出名了，有钱了，就该贪图享受了，无论好的坏的都要...,1518
6,6,娱乐,独家专访李小璐：我与杨晓芸完全不同 ［提要］ ６月１１日早上１０点，上海电视节搜狐娱乐现场演...,1457
7,7,娱乐,《二十四城记》仍募捐放映 贾樟柯成人气明星（图） 更多图片３００位观众，２２７５９元善款——...,1666
8,8,娱乐,沈阳日报：别把“新红楼”弄成“烂尾楼” 新版《红楼梦》说２００７年１０月开机，拖到今年３月，...,1839
9,9,娱乐,女编剧讨薪案升级为多方混战 出品方反告制片方 女编剧王伊向电视剧《牟氏庄园》出品方讨薪案，前...,1599


In [6]:
train_pdf.describe()

,Unnamed: 0,分类,文章,字数
count,23549,23549,23549,23549
unique,23549,12,18400,5477
top,23935,健康,同学金榜题名同学在今年的中国高考，以及整个高中学习过程中，取得了极其优异的成绩。全球顶级人...,1278
freq,1,2000,109,119


In [7]:
test_pdf.head(10)

,分类,文章
0,娱乐,组图：黄健翔拍时装大片 承认口无遮拦 ２００６年之前，他只是最好的体育节目主持人之一。２００...
1,娱乐,奥运明星写真集锦曝光 展现健康时尚（图） 来源：人民网奥运明星奥运明星大满贯――属于最强者的...
2,娱乐,内地票房榜：《功夫熊猫》获全胜 带动内地影市 《功夫熊猫》首映周末逼４０００万２００６年１月...
3,娱乐,编者按： 昨天，央视紧急停播动画片《虹猫蓝兔七侠传》事件经报道后，引发了数十万网民的热烈大辩...
4,娱乐,第十一届上海国际电影节 金爵奖评委名单 ［点击图片进入下一页］金爵奖评委陈冲陈冲（美籍华裔女...
5,娱乐,５．１２责任中国：央视汶川地震直播令世界刮目相看 “数秒”为总理讲话抢出“第一时间” 专机上...
6,娱乐,《画皮》片花独家曝光 周迅剥掉整张人皮（图） 周迅在片中的造型赵薇在片中的造型四十年前，老版...
7,娱乐,影坛奇人第二期 陈勋奇：香港最后一位电影全才 ［前言］ 陈勋奇是公认的香港电影全才。简单点可...
8,娱乐,《全民超人》首映在即 “超人”竟此丢人 ７月２日，《全民超人》将在全球同步首映。日前，这部备...
9,娱乐,《全民超人》７月２日上映 这个“超人”很麻烦 ７月２日，《全民超人》全球首映，国内也将于当天...


In [8]:
test_pdf.describe()

,分类,文章
count,12000,12000
unique,12,10367
top,女人,同学金榜题名同学在今年的中国高考，以及整个高中学习过程中，取得了极其优异的成绩。全球顶级人...
freq,1000,58


# 文章分词处理 pipeline

In [10]:
import re
import os
import numpy as np
import pandas as pd
import jieba
import jieba.analyse  

#打开stopwords文件  r"""D:\code\python\csdn_nlp\NLP_DEMO\Resources\stopwords\all_stopwords.txt"""
def open_stop(file_stop):
    stopwords = [line.strip() for line in open(file_stop, 'r', encoding='utf-8').readlines()]
    return stopwords

stopwords = open_stop(r"""D:\code\python\csdn_nlp\NLP_DEMO\Resources\stopwords\cn_stopwords.txt""")


# 针对train文件每行（每篇文章）处理的 pipeline，返回汉语分词列表
def extract_words_one_line(str_line):
    
    chinesewords_sentence = []

    #只保留中文字符
    def remove(text):
        remove_chars = r'[^\u4e00-\u9fa5]'
        return re.sub(remove_chars, '', text)

    #利用jieba进行分词
    def seg_sentence(sentence):
        sentence_seged = jieba.cut(sentence.strip())
        outstr = ''
        for word in sentence_seged:
            if word not in stopwords:
                if word != '\t':
                    chinesewords_sentence.append(word)

        return chinesewords_sentence


    inputs = str_line
    #获取每一句中的分词
    words_in_sentence = []
   
    line_delete = remove(inputs)
    line_seg = seg_sentence(line_delete)
    

    return chinesewords_sentence
    

In [11]:
# 调用样例
extract_words_one_line("今天天气不错，挺风和日丽的.我们下午没有课,sometime")       

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.801 seconds.
Prefix dict has been built successfully.


['今天天气', '不错', '挺', '风和日丽', '下午', '没有', '课']

# 并行分词处理

并行分词处理有两种方案：

1. 参考 https://zhuanlan.zhihu.com/p/132669978 并行化apply 加速，使用第三方包：pandarallel ，挺好用但是 windows 不支持

2. 采用第二种方法 手写并行处理后拼接dataframe

from joblib import Parallel,delayed
参考：https://blog.csdn.net/Jerr__y/article/details/71425298?utm_source=blogxgwz1

In [13]:
import time
from joblib import Parallel,delayed
def tmp_func(df1):
    df1['分词'] = df1['文章'].apply(extract_words_one_line)
    return df1

def apply_parallel(df_grouped,func):
    results = Parallel(n_jobs=8)(delayed(func)(group) for name,group in df_grouped)
    return pd.concat(results)



time0 = time.time()
#  训练集
df_grouped = train_pdf.groupby(train_pdf['分类'])
train_df_all = apply_parallel(df_grouped, tmp_func)

#测试集
df_grouped = test_pdf.groupby(test_pdf['分类'])
test_df_all = apply_parallel(df_grouped, tmp_func)

print ('time costed {0:.2f}'.format(time.time() - time0))

#train_pdf['分词'] = train_pdf['文章'].apply(extract_words_one_line)

time costed 433.05


In [14]:
train_df_all.head(10)

,Unnamed: 0,分类,文章,字数,分词
9669,10000,体育,欧洲杯观察：足坛格局地震 东南欧已能占半边天 搜狐体育讯 土耳其与克罗地亚在本届欧洲杯１／４...,1818,"[欧洲杯, 观察, 足坛, 格局, 地震, 东南欧, 已能, 占, 半边天, 搜狐, 体育讯..."
9670,10001,体育,来源：搜狐体育 作者：雷欧 主队 比分 客队 进球视频青岛 １－１ 武汉浙江 １－１ 北京搜...,2190,"[来源, 搜狐, 体育, 作者, 雷欧, 主队, 比分, 客队, 进球, 视频, 青岛, 武..."
9671,10002,体育,欧洲杯－帕夫柳琴科范尼破门 荷兰１－１俄罗斯加时 搜狐体育讯 北京时间６月２２日，２００８欧...,2078,"[欧洲杯, 帕夫, 柳琴, 科范尼, 破门, 荷兰, 俄罗斯, 加时, 搜狐, 体育讯, 北..."
9672,10003,体育,郅联璧合配合日臻完善 斯杯之前尤纳斯再谈裁员 ７年后重登上海滩，中国男篮来去匆匆。队长刘炜回...,1810,"[郅, 联璧, 合, 配合, 日臻完善, 斯杯, 之前, 尤纳斯, 再谈, 裁员, 年, 重..."
9673,10004,体育,盘踞世界第一逼近五百周 伍兹创造高尔夫里程碑 搜狐体育讯 下周一，老虎－伍兹将第４９９周稳坐...,1893,"[盘踞, 世界, 第一, 逼近, 五百, 周伍, 兹, 创造, 高尔夫, 里程碑, 搜狐, ..."
9674,10005,体育,第五日看点：郑洁挑战ＮＯ．１ 费天王冲击男单１６强 搜狐体育讯 北京时间６月２７日晚，２００...,2082,"[第五日, 看点, 郑洁, 挑战, 费, 天王, 冲击, 男单, 强, 搜狐, 体育讯, 北..."
9675,10007,体育,王德显改变双姝命运 邢慧娜孙英杰成败均因一人 ２００７年年底，邢慧娜来到了北京。田管中心已经...,1855,"[王德显, 改变, 双姝, 命运, 邢慧娜, 孙, 英杰, 成败, 均, 一人, 年, 年底..."
9676,10008,体育,半场实录：皮尔斯异军突起 湖人队领先凯尔特人 搜狐直播员：搜狐网友：赛式是２３２制的，还是主...,8550,"[半场, 实录, 皮尔斯, 异军突起, 湖人队, 领先, 凯尔特人, 搜狐, 直播员, 搜狐..."
9677,10009,体育,温格：德国可成英格兰榜样 三狮军团差在意志力 晨报特派记者 甘慧（奥地利维也纳６月２９日电）...,2162,"[温格, 德国, 可成, 英格兰, 榜样, 三狮, 军团, 差, 意志力, 晨报, 特派记者..."
9678,10010,体育,欧洲杯－范尼斯内德建功 荷兰３－０狂屠意大利 搜狐体育讯 北京时间６月１０日凌晨２：４５，２...,2643,"[欧洲杯, 范, 尼斯, 德, 建功, 荷兰, 狂屠, 意大利, 搜狐, 体育讯, 北京, ..."


In [15]:
test_df_all.head(10)

,分类,文章,分词
5000,体育,奥运男女水球赛程公布 中国队面临强敌形势严峻 奥运官方网站讯 北京奥运会水球项目详细赛程近日...,"[奥运, 男女, 水球赛, 程, 公布, 中国队, 面临, 强敌, 形势严峻, 奥运, 官方..."
5001,体育,世界冠军惨败另有内因 罕见怪象揭示意甲已衰败 搜狐体育讯 要在世界杯上封王，首当其冲的当然是...,"[世界冠军, 惨败, 另有, 内因, 罕见, 怪象, 揭示, 意甲, 衰败, 搜狐, 体育讯..."
5002,体育,世界风杯配对赛业余组成绩 侯波四杆优势居第一 搜狐体育讯 ６月１日下午，２００８年“世界风杯...,"[世界, 风杯, 配对, 赛, 业余组, 成绩, 侯波, 四杆, 优势, 居, 第一, 搜狐..."
5003,体育,凯尔特人晋级一波三折 ２１年后首度进总决赛（图） 北京时间５月３１日，凯尔特人在东部决赛第六...,"[凯尔特人, 晋级, 一波三折, 年, 首度, 进, 总决赛, 图, 北京, 时间, 月, ..."
5004,体育,绿衫军王朝回顾：红衣主教未卜先知 湖人最悲情 搜狐体育讯 不知道“红衣主教”奥尔巴赫的雪茄香...,"[绿衫, 军, 王朝, 回顾, 红衣主教, 未卜先知, 湖人, 悲情, 搜狐, 体育讯, 知..."
5005,体育,十年总决赛恩怨：绿军八胜占优 魔术师托起湖人 搜狐体育讯北京时间６月６日上午，ＮＢＡ历史上最...,"[十年, 总决赛, 恩怨, 绿军, 八胜, 占优, 魔术师, 托起, 湖人, 搜狐, 体育讯..."
5006,体育,最新动态 申花２００７中超赛程轮次 期 主队１ ３．３ 申花０－２北京２ ３．１１ 申花１－...,"[最新, 动态, 申花, 中超, 赛程, 轮次, 期, 主队, 申花, 北京, 申花, 山东..."
5007,体育,欧洲杯失意球星疗伤法：Ｃ罗携女友海滩度假（图） 点击查看其它图片俗话说“吃不到葡萄就说葡萄酸...,"[欧洲杯, 失意, 球星, 疗伤, 法罗, 携, 女友, 海滩, 度假, 图, 点击, 查看..."
5008,体育,国足进一球实现有难度 １８人大名单成最高级机密 今晚八时，国足将在天津奥体中心迎来最重要的一...,"[国足, 进一球, 实现, 难度, 人大, 名单, 成, 最高级, 机密, 今晚, 八时, ..."
5009,体育,乒球故事：容国团奠基国乒王朝 ８１年成功进奥运 以下是《乒乓世界》杂志２００８年第６期封面故...,"[乒球, 故事, 容国团, 奠基, 国乒, 王朝, 年, 成功, 进, 奥运, 以下, 乒乓..."


In [16]:
test = test_df_all.groupby(test_df_all['分类'])
test.describe()

文章                                                                 \
    count unique                                                top freq   
分类                                                                         
体育   1000    840  小记者：义结兄弟情谊　ａｄｉｄａｓ夏季联赛广州站见 ［点击图片进入下一页］２００８年５月３１...    6   
健康   1000    903  地坛医院新院落成　奥运之后新院全面启用 ６月２７日上午备受北京市民关注的北京地坛医院新址落成...    4   
女人   1000    694  如何检验男人是否真是“第一次” 男人们在初次与女性发生性关系时，事先要无数次地在脑海中进行模...    7   
娱乐   1000    744  周笔畅撰文“无因暴怒”　痛斥灾区作秀明星 可爱的、憨憨的“笔笔”——周笔畅——如你所知的关于...   28   
房地产  1000    909  业内论坛 版主：　毕普　ｔａｌｋａｔｉｖｅ　也是另类　华沉思　周海山　成业　醋咪　ｚｈａｍｕ...    7   
教育   1000    812   同学金榜题名同学在今年的中国高考，以及整个高中学习过程中，取得了极其优异的成绩。全球顶级人...   58   
文化   1000    880  “川人从未负国”真相：四川曾两次举省殉国 四川大地震后，流传着这样一句话，“川人从未负国，国...    5   
新闻   1000    880  汶川地震感动我们的故事：立体大营救（组图） 抗震救灾官兵在废墟上争分夺秒抢救生命。沈玲摄连日...    5   
旅游   1000    918  在越南赌场得到美女的丝袜［组图］ ［点击图片进入下一页］８月１７号，我和父亲随旅游团到陌生的...    6   
汽车   1000    955  看谁最有冠军相　夏季明星新车捉对比拼 夏季车市将被新车浪潮席卷，预计整个第二季度将会有多达２...    3   
科技   1000    930  常小兵会晤分析师　中国联通高价卖Ｃ网另有玄机 本报记者　罗诺６月３日下午３点半，位于北京西单...    3   
财经   1000    902  李德水：股市完全可以实现稳定健康发展 李德水强调，应全面准确理解宏观调控政策。李江松／ＣＦＰ...    5   

       分词                                                                 
    count unique                                                top freq  
分类                                                                        
体育   1000    838  [三, 巨头, 王者, 归来, 杀退, 科比, 绿军胜, 湖人破, 年, 坚冰, 搜狐, 体...    6  
健康   1000    902  [地坛, 医院, 新, 院落, 成, 奥运, 之后, 新, 院, 全面, 启用, 月, 日,...    4  
女人   1000    688  [金城武, 首位, 亚洲, 男士, 代言, 年, 月, 讯, 欣然, 宣布, 亚洲, 著名,...    7  
娱乐   1000    742  [周笔畅, 撰文, 无因, 暴怒, 痛斥, 灾区, 作秀, 明星, 可爱, 憨憨的, 笔笔,...   28  
房地产  1000    871  [业内, 论坛, 版主, 毕普, 另类, 华, 沉思, 周, 海山, 成业, 醋, 咪, 老...   16  
教育   1000    811  [同学, 金榜题名, 同学, 今年, 中国, 高考, 整个, 高中, 学习, 过程, 中, ...   58  
文化   1000    874  [川人, 从未, 负国, 真相, 四川, 两次, 举省, 殉国, 四川, 地震, 流传, 一...    5  
新闻   1000    877  [汶川, 地震, 感动, 故事, 立体, 营救, 组图, 抗震救灾, 官兵, 废墟, 争分夺...    5  
旅游   1000    918  [越南, 赌场, 得到, 美女, 丝袜, 组图, 点击, 图片, 进入, 一页, 月, 号,...    6  
汽车   1000    955  [万元, 之间, 款, 紧凑型, 都市, 当前, 中国, 紧凑型, 市场, 百家争鸣, 形容...    3  
科技   1000    929  [三位, 高管, 高位, 套现, 腾讯, 遇, 拐点, 见习, 记者, 程久龙, 回归, 股...    3  
财经   1000    902  [李德水, 股市, 完全, 实现, 稳定, 健康, 发展, 李德水, 强调, 应, 全面, ...    5

In [17]:
# 重新定义标签结果

In [18]:
#定义标签替换字典

label2category = {
0:'财经',
1:'房地产',
2:'健康',
3:'教育',
4:'科技',
5:'旅游',
6:'女人',
7:'汽车',
8:'体育',
9:'文化',
10:'新闻',
11:'娱乐'
}
category2label = dict(zip(label2category.values(), label2category.keys()))
print(category2label)

train_df_all['label'] = train_df_all['分类'].replace(category2label)
test_df_all['label'] = test_df_all['分类'].replace(category2label)

train_df_all = train_df_all.reset_index(drop=True)
test_df_all = test_df_all.reset_index(drop=True)

{'财经': 0, '房地产': 1, '健康': 2, '教育': 3, '科技': 4, '旅游': 5, '女人': 6, '汽车': 7, '体育': 8, '文化': 9, '新闻': 10, '娱乐': 11}


In [19]:
#保存中间结果以防丢失

train_df_all.to_csv('train2.csv', encoding="utf-8", index=False)
test_df_all.to_csv('test2.csv', encoding="utf-8", index=False)

In [20]:
# Word2Vec模型



# 使用gensim 会出现各种问题： 以下原则：

（1）先用pip uninstall +包名命令依次将numpy，scipy，gensim卸载。 

（2）再按numpy，scipy，gensim顺序依次安装，能够解决这一问题。 


pip install numpy -i https://pypi.douban.com/simple 

pip install scipy -i https://pypi.douban.com/simple 

pip install gensim -i https://pypi.douban.com/simple




In [21]:
import gensim
from gensim.models import Word2Vec
#导入数据

time0 = time.time()

sent_words = train_df_all['分词'].values.tolist()

# 参数说明 https://blog.csdn.net/u011748542/article/details/85880852
# 训练维数 128 维
my_model = gensim.models.Word2Vec(sent_words, sg=0, size=128, window=3,iter=5,min_count=3, negative=3, sample=0.001, hs=1)
#模型保存
my_model.wv.save_word2vec_format('./word2vec_model.txt', binary=False)
print ('time costed {0:.2f}'.format(time.time() - time0))

time costed 229.38


In [22]:
import gensim
import numpy as np

#导入数据
time0 = time.time()
# 首先初始化一个word2vec 模型：
w2v_model = gensim.models.Word2Vec(size=300, window = 3,sg=1, min_count=3)
w2v_model.build_vocab(sent_words)
# 再加载第三方预训练模型：
third_model = gensim.models.KeyedVectors.load_word2vec_format(r'./sgns.sogounews.bigram-char/sgns.sogounews.bigram-char', binary=False)
# 通过 intersect_word2vec_format()方法merge词向量：
w2v_model.build_vocab([list(third_model.vocab.keys())], update=True)
w2v_model.intersect_word2vec_format(r'./sgns.sogounews.bigram-char/sgns.sogounews.bigram-char', binary=False, lockf=1.0)
w2v_model.train(sent_words, total_examples=w2v_model.corpus_count, epochs=5)

print("Model training finished.")

w2v_model.wv.save_word2vec_format('./word2vec_ensemble.txt', binary=False)
print("Model saved.")
print ('time costed {0:.2f}'.format(time.time() - time0))

Model training finished.
Model saved.
time costed 659.90


# svm 进行训练


将每篇文章的词向量融合成一个向量，使用计算平均词向量来表示文档的方法。
将文章内包含的所有词的向量相加再取平均，这样，我们就可以得到一个和词向量维度相同的指示文章信息的向量。

In [23]:
import numpy as np
#num_features表示的文本单词维度大小

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('./word2vec_ensemble.txt',binary=False)

def average_word_vectors(words,model,vocabulary,num_features):
    feature_vector=np.zeros((num_features,),dtype='float64')
    nwords=0
    for word in words:
        if word in vocabulary:
            nwords=nwords+1
            feature_vector=np.add(feature_vector,model[word])
    if nwords:
        feature_vector=np.divide(feature_vector,nwords)
    return feature_vector

def averaged_word_vectorizer(corpus,model,num_features):
    #get the all vocabulary
    vocabulary=set(model.wv.index2word)
    features=[average_word_vectors(tokenized_sentence,model,vocabulary,num_features) for tokenized_sentence in corpus]
    return np.array(features)

def get_word_vectors(data):
    words_art=[]
    for i in range(len(data)):
        words_art.append((data.loc[i]))
    return averaged_word_vectorizer(words_art,model=w2v_model,num_features=300)

In [24]:
train_df_all['分词'].loc[0]
print(type(train_df_all['分词'].loc[0]))
print( range(len(train_df_all['分词'])))

<class 'list'>
range(0, 23549)


In [25]:
X_train = get_word_vectors(train_df_all['分词'])
y_train = train_df_all.label

X_test = get_word_vectors(test_df_all['分词'])
y_test = test_df_all.label

<ipython-input-23-0586e5840f31>:19: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  vocabulary=set(model.wv.index2word)


In [27]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

time0 = time.time()
clf = svm.SVC()
grid_values = {'gamma': [0.001, 0.01, 0.05, 0.1, 1, 10],
               'C':[0.01, 0.1, 1, 10, 100]}


grid_clf = GridSearchCV(clf, param_grid = grid_values,scoring = 'f1_macro')
grid_clf.fit(X_train, y_train)
y_grid_pred = grid_clf.predict(X_test)

print('Test set F1: ', f1_score(y_test,y_grid_pred,average='macro'))
print('Grid best parameter (max. f1): ', grid_clf.best_params_)
print('Grid best score (accuracy): ', grid_clf.best_score_)
print ('time costed {0:.2f}'.format(time.time() - time0))

Test set F1:  0.8751931335464355
Grid best parameter (max. f1):  {'C': 10, 'gamma': 1}
Grid best score (accuracy):  0.8674631170018857
time costed 12607.92


In [29]:
import joblib

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

time0 = time.time()

clf = svm.SVC(C=10, gamma= 1,probability=True).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
joblib.dump(clf, "download_w2v_svm.m")
print ('time costed {0:.2f}'.format(time.time() - time0))

              precision    recall  f1-score   support

           0       0.76      0.77      0.76      1000
           1       0.92      0.89      0.90      1000
           2       0.94      0.91      0.92      1000
           3       0.94      0.92      0.93      1000
           4       0.88      0.86      0.87      1000
           5       0.92      0.86      0.89      1000
           6       0.90      0.91      0.91      1000
           7       0.96      0.93      0.94      1000
           8       0.99      0.97      0.98      1000
           9       0.73      0.83      0.78      1000
          10       0.71      0.75      0.73      1000
          11       0.89      0.89      0.89      1000

    accuracy                           0.87     12000
   macro avg       0.88      0.87      0.88     12000
weighted avg       0.88      0.87      0.88     12000

time costed 516.14


In [ ]:
import joblib

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

time0 = time.time()

clf = svm.SVC(C=100, gamma= 1,probability=True).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
joblib.dump(clf, "download_w2v_svm10.m")
print ('time costed {0:.2f}'.format(time.time() - time0))